<a href="https://colab.research.google.com/github/Talizg03/weather/blob/main/30%2C1%2C25_isolationforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
import pandas as pd
uploaded = files.upload()

df = pd.read_csv("Train.csv")
#print(df.head())
df.head()

Saving Train.csv to Train (1).csv


<ipython-input-5-1280e8b6c072>:5: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Train.csv")


SalesID  SalePrice  MachineID  ModelID  datasource  auctioneerID  YearMade  \
0  1139246      66000     999089     3157         121           3.0      2004   
1  1139248      57000     117657       77         121           3.0      1996   
2  1139249      10000     434808     7009         121           3.0      2001   
3  1139251      38500    1026470      332         121           3.0      2001   
4  1139253      11000    1057373    17311         121           3.0      2007   

   MachineHoursCurrentMeter UsageBand         saledate  ...  \
0                      68.0       Low  11/16/2006 0:00  ...   
1                    4640.0       Low   3/26/2004 0:00  ...   
2                    2838.0      High   2/26/2004 0:00  ...   
3                    3486.0      High   5/19/2011 0:00  ...   
4                     722.0    Medium   7/23/2009 0:00  ...   

  Undercarriage_Pad_Width Stick_Length Thumb Pattern_Changer Grouser_Type  \
0                     NaN          NaN   NaN             NaN          NaN   
1                     NaN          NaN   NaN             NaN          NaN   
2                     NaN          NaN   NaN             NaN          NaN   
3                     NaN          NaN   NaN             NaN          NaN   
4                     NaN          NaN   NaN             NaN          NaN   

  Backhoe_Mounting Blade_Type Travel_Controls Differential_Type  \
0              NaN        NaN             NaN          Standard   
1              NaN        NaN             NaN          Standard   
2              NaN        NaN             NaN               NaN   
3              NaN        NaN             NaN               NaN   
4              NaN        NaN             NaN               NaN   

  Steering_Controls  
0      Conventional  
1      Conventional  
2               NaN  
3               NaN  
4               NaN  

[5 rows x 53 columns]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   401125 non-null  int64  
 1   SalePrice                 401125 non-null  int64  
 2   MachineID                 401125 non-null  int64  
 3   ModelID                   401125 non-null  int64  
 4   datasource                401125 non-null  int64  
 5   auctioneerID              380989 non-null  float64
 6   YearMade                  401125 non-null  int64  
 7   MachineHoursCurrentMeter  142765 non-null  float64
 8   UsageBand                 69639 non-null   object 
 9   saledate                  401125 non-null  object 
 10  fiModelDesc               401125 non-null  object 
 11  fiBaseModel               401125 non-null  object 
 12  fiSecondaryDesc           263934 non-null  object 
 13  fiModelSeries             56908 non-null   o

In [ ]:

import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

df = pd.read_csv("Train.csv")
X, y = df.drop(columns=['SalePrice']), df['SalePrice']
X = pd.get_dummies(X, drop_first=True)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the Isolation Forest model
iso_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
iso_forest.fit(X_train)

# Predict anomaly scores and labels on a copy of X_test
X_test_copy = X_test.copy()
df['anomaly_score'] = iso_forest.decision_function(X)  # Quantitative weirdness
df['anomaly'] = iso_forest.predict(X)  # Binary anomaly label
df['anomaly_label'] = df['anomaly'].map({1: 'Normal', -1: 'Anomaly'})

# Plot carat vs. price with hue as anomaly score (quantitative weirdness)
plt.figure(figsize=(12, 8))
scatter = plt.scatter(
    df['ModelID'], df['SalePrice'], c=df['anomaly_score'], cmap='coolwarm',
    # edgecolor='k',
    alpha=0.8
)
plt.colorbar(scatter, label='Anomaly Score (Weirdness)')
plt.title('Carat vs. Price with Anomaly Score as Hue')
plt.suptitle('Anomaly Score: lower is more anomalous')
plt.xlabel('ModelID')
plt.ylabel('SalePrice')
plt.grid(True)
plt.show()


<ipython-input-1-280ed2aa1b3c>:7: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Train.csv")


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor()
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print("Test RMSE:", mean_squared_error(y_test, y_test_pred) ** 0.5)

NameError: name 'X_train' is not defined

In [ ]:
df2 = df[df.anomaly==1].copy()
X2 = df2.drop(columns=['SalePrice', 'anomaly_label', 'anomaly', 'anomaly_score'])
X2 = pd.get_dummies(X2, drop_first=True)
y2 = df2['SalePrice']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

model2 = RandomForestRegressor()
model2.fit(X2, y2)
y2_test_pred = model2.predict(X2_test)
print("Test RMSE:", mean_squared_error(y2_test, y2_test_pred) ** 0.5)

In [ ]:
df.sort_values("anomaly_score").sample(20)

In [ ]:
pd.Series(model2.feature_importances_, index=X2_train.columns).sort_values(ascending=False)